In [ ]:
#This file shows Word2Vec, Doc2Vec, Glove, and various models implemented on them after oversampling is done on the dataframe

###  Importing libraries and preprocessing ###

In [ ]:
import numpy as np
np.random.seed(42) # results are reproducible and comparable across different runs

In [ ]:
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
CMD = ConfusionMatrixDisplay
from sklearn.metrics import classification_report
CR = classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import gensim
import spacy
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [ ]:
df = pd.read_csv('/content/fake_papers_train_part_public.csv')
df = df.drop(columns = ['id'], axis = 1)
print(df.shape)
df.head()

(5350, 2)


,text,fake
0,Modern two-dimensional imaging is of such qual...,0
1,Background: The optimal sequence of systemic p...,1
2,This chapter opens with a discussion of the ef...,1
3,The time scale of the ultra-short-term can str...,1
4,Electronic nose or machine olfaction are syste...,1


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = ''.join(c for c in text if c.isalpha() or c.isspace())
    doc = nlp(text)
    text = [token.lemma_ for token in doc]
    return ' '.join(text)

# Apply the clean_text function to each row of the 'text' column in the dataframe
df['cleanedtext'] = df['text'].apply(lambda x: clean_text(x.lower()))

# Create a new column to store the lists of cleaned text for each document
df['cleanedtext_list'] = None

# Loop through each row in the dataframe and store the cleaned text for each document in a list
for index, row in df.iterrows():
    cleaned_text_list = row['cleanedtext'].split()
    df.at[index, 'cleanedtext_list'] = cleaned_text_list
df = df.drop('cleanedtext', axis=1)

# Print the first 5 rows of the dataframe to check the new column
df.head(5)

,text,fake,cleanedtext_list
0,Modern two-dimensional imaging is of such qual...,0,"[modern, twodimensional, imaging, be, of, such..."
1,Background: The optimal sequence of systemic p...,1,"[background, the, optimal, sequence, of, syste..."
2,This chapter opens with a discussion of the ef...,1,"[this, chapter, open, with, a, discussion, of,..."
3,The time scale of the ultra-short-term can str...,1,"[the, time, scale, of, the, ultrashortterm, ca..."
4,Electronic nose or machine olfaction are syste...,1,"[electronic, nose, or, machine, olfaction, be,..."


In [ ]:
df['fake'].value_counts()

1    3664
0    1686
Name: fake, dtype: int64

In [ ]:
X = list(df['cleanedtext_list'])
y = list(df['fake'])

In [ ]:
# Split the data into training and test sets
X_val_train, X_test, y_val_train, y_test = train_test_split(df['cleanedtext_list'], df['fake'], test_size=0.2,random_state=42)

In [ ]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Word2Vec and Doc2Vec

Random Forest on Word2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Train a Skip Gram Word2Vec model on the training data
w2v_model = gensim.models.Word2Vec(X_val_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2,
                                   workers=4,
                                   sg=1)  # set sg=1 to use the Skip Gram algorithm

words = set(w2v_model.wv.index_to_key)

# Vectorize the training and test data using the trained Skip Gram Word2Vec model
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])

# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Fit the model to the oversampled training data
rf_model = rf.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)

<ipython-input-49-9986bacf82f4>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-49-9986bacf82f4>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])




Precision: 0.986 / Recall: 0.974 / Accuracy: 0.973


Confusion Matrix:
[[324  10]
 [ 19 717]]


Random forest on Doc2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.model_selection import train_test_split

# Convert the training data to TaggedDocuments for Doc2Vec
tagged_docs_train = [TaggedDocument(words=words, tags=[idx]) for idx, words in enumerate(X_val_train)]

# Train a Doc2Vec model on the training data
d2v_model = Doc2Vec(tagged_docs_train,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# Vectorize the training and test data using the trained Doc2Vec model
X_train_vect = np.array([d2v_model.infer_vector(words) for words in X_val_train])
X_test_vect = np.array([d2v_model.infer_vector(words) for words in X_test])

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect, y_train_np)

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Fit the model to the oversampled training data
rf_model = rf.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.91 / Recall: 0.938 / Accuracy: 0.893


Confusion Matrix:
[[266  68]
 [ 46 690]]


SVM on Word2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Train a Skip Gram Word2Vec model on the training data
w2v_model = gensim.models.Word2Vec(X_val_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2,
                                   workers=4,
                                   sg=1)  # set sg=1 to use the Skip Gram algorithm

words = set(w2v_model.wv.index_to_key)

# Vectorize the training and test data using the trained Skip Gram Word2Vec model
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])

# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create an SVM classifier
svm = SVC()

# Fit the model to the oversampled training data
svm_model = svm.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = svm_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)

<ipython-input-50-c00b2bd39275>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-50-c00b2bd39275>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])




Precision: 0.937 / Recall: 0.772 / Accuracy: 0.807


Confusion Matrix:
[[296  38]
 [168 568]]


SVM on Doc2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Train a Doc2Vec model on the training data
tagged_documents = [TaggedDocument(words=words, tags=[idx]) for idx, words in enumerate(X_val_train)]
d2v_model = Doc2Vec(tagged_documents,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# Vectorize the training and test data using the trained Doc2Vec model
X_train_vect = np.array([d2v_model.infer_vector(words) for words in X_val_train])
X_test_vect = np.array([d2v_model.infer_vector(words) for words in X_test])

# Convert the data to numpy arrays
X_train_vect = np.array(X_train_vect)
X_test_vect = np.array(X_test_vect)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect, y_train_np)

# Create an SVM classifier
svm = SVC()

# Fit the model to the oversampled training data
svm_model = svm.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = svm_model.predict(X_test_vect)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.925 / Recall: 0.857 / Accuracy: 0.854


Confusion Matrix:
[[283  51]
 [105 631]]


Decision Tree on Word2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Train a Word2Vec model on the training data
w2v_model = gensim.models.Word2Vec(X_val_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2,
                                   workers=4)

words = set(w2v_model.wv.index_to_key)

# Vectorize the training and test data using the trained Word2Vec model
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])

# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Decision Tree classifier
dt = DecisionTreeClassifier()

# Fit the model to the oversampled training data
dt_model = dt.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = dt_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)

<ipython-input-53-ae3a08045676>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-53-ae3a08045676>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])




Precision: 0.969 / Recall: 0.966 / Accuracy: 0.955


Confusion Matrix:
[[311  23]
 [ 25 711]]


Desicion Tree on Doc2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Train a Doc2Vec model on the training data
tagged_documents = [TaggedDocument(words=words, tags=[idx]) for idx, words in enumerate(X_val_train)]
d2v_model = Doc2Vec(tagged_documents,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# Vectorize the training and test data using the trained Doc2Vec model
X_train_vect = np.array([d2v_model.infer_vector(words) for words in X_val_train])
X_test_vect = np.array([d2v_model.infer_vector(words) for words in X_test])

# Convert the data to numpy arrays
X_train_vect = np.array(X_train_vect)
X_test_vect = np.array(X_test_vect)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect, y_train_np)

# Create a Decision Tree classifier
dt = DecisionTreeClassifier()

# Fit the model to the oversampled training data
dt_model = dt.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = dt_model.predict(X_test_vect)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.844 / Recall: 0.787 / Accuracy: 0.753


Confusion Matrix:
[[227 107]
 [157 579]]


MLP Classifier on Word2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Train a Word2Vec model on the training data
w2v_model = gensim.models.Word2Vec(X_val_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2,
                                   workers=4)

words = set(w2v_model.wv.index_to_key)

# Vectorize the training and test data using the trained Word2Vec model
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])

# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a 1-layer MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam')

# Fit the model to the oversampled training data
mlp_model = mlp.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = mlp_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)

<ipython-input-54-7fe373e4ba54>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-54-7fe373e4ba54>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])




Precision: 0.907 / Recall: 0.849 / Accuracy: 0.836


Confusion Matrix:
[[270  64]
 [111 625]]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier on Doc2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

# Convert the training data to TaggedDocuments for Doc2Vec
tagged_docs_train = [TaggedDocument(words=words, tags=[idx]) for idx, words in enumerate(X_val_train)]

# Train a Doc2Vec model on the training data
d2v_model = Doc2Vec(tagged_docs_train,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# Vectorize the training and test data using the trained Doc2Vec model
X_train_vect = np.array([d2v_model.infer_vector(words) for words in X_val_train])
X_test_vect = np.array([d2v_model.infer_vector(words) for words in X_test])

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect, y_train_np)

# Create a 1-layer MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', solver='adam')

# Fit the model to the oversampled training data
mlp_model = mlp.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = mlp_model.predict(X_test_vect)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.934 / Recall: 0.943 / Accuracy: 0.915


Confusion Matrix:
[[285  49]
 [ 42 694]]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Word2Vec using Logistic Regression

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Train a Skip Gram Word2Vec model on the training data
w2v_model = gensim.models.Word2Vec(X_val_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2,
                                   workers=4,
                                   sg=1)  # set sg=1 to use the Skip Gram algorithm

words = set(w2v_model.wv.index_to_key)

# Vectorize the training and test data using the trained Skip Gram Word2Vec model
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])

# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Logistic Regression classifier
lr = LogisticRegression()

# Fit the model to the oversampled training data
lr_model = lr.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = lr_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)

<ipython-input-61-2dbeff1e3b4e>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-61-2dbeff1e3b4e>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])




Precision: 0.884 / Recall: 0.736 / Accuracy: 0.752


Confusion Matrix:
[[263  71]
 [194 542]]


Logistic Regression on Doc2Vec

In [ ]:
import gensim
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.model_selection import train_test_split

# Convert the training data to TaggedDocuments for Doc2Vec
tagged_docs_train = [TaggedDocument(words=words, tags=[idx]) for idx, words in enumerate(X_val_train)]

# Train a Doc2Vec model on the training data
d2v_model = Doc2Vec(tagged_docs_train,
                    vector_size=100,
                    window=5,
                    min_count=2,
                    workers=4)

# Vectorize the training and test data using the trained Doc2Vec model
X_train_vect = np.array([d2v_model.infer_vector(words) for words in X_val_train])
X_test_vect = np.array([d2v_model.infer_vector(words) for words in X_test])

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect, y_train_np)

# Create a Logistic Regression classifier
lr = LogisticRegression()

# Fit the model to the oversampled training data
lr_model = lr.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = lr_model.predict(X_test_vect)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.87 / Recall: 0.766 / Accuracy: 0.761


Confusion Matrix:
[[250  84]
 [172 564]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


###  Glove

In [ ]:
import gensim.downloader as api

# Load the pre-trained GloVe model
model = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

import numpy as np
from tqdm import tqdm

--2023-05-15 16:53:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-05-15 16:53:24--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-05-15 16:53:24--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
def load_glove_embeddings(path):
    embeddings_index = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

glove_path = '/content/glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_path)

In [ ]:
# Vectorize the training and test data using GloVe embeddings
X_train_vect = np.array([np.array([glove_embeddings.get(i, np.zeros(100)) for i in ls if i in words])
                         for ls in X_val_train])
X_test_vect = np.array([np.array([glove_embeddings.get(i, np.zeros(100)) for i in ls if i in words])
                        for ls in X_test])

<ipython-input-67-1083dbf9d124>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([glove_embeddings.get(i, np.zeros(100)) for i in ls if i in words])
<ipython-input-67-1083dbf9d124>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([glove_embeddings.get(i, np.zeros(100)) for i in ls if i in words])


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

Glove using RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Average the word vectors for each sentence (and assign a vector of zeros if no embedding is found)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Fit the model to the oversampled training data
rf_model = rf.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.873 / Recall: 0.978 / Accuracy: 0.887


Confusion Matrix:
[[229 105]
 [ 16 720]]


Glove using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Average the word vectors for each sentence (and assign a vector of zeros if no embedding is found)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Logistic Regression classifier
lr = LogisticRegression()

# Fit the model to the oversampled training data
lr_model = lr.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = lr_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.752 / Recall: 0.769 / Accuracy: 0.666


Confusion Matrix:
[[147 187]
 [170 566]]


Desicion Tree using Glove

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Average the word vectors for each sentence (and assign a vector of zeros if no embedding is found)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create a Decision Tree classifier
dt = DecisionTreeClassifier()

# Fit the model to the oversampled training data
dt_model = dt.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = dt_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.87 / Recall: 0.97 / Accuracy: 0.879


Confusion Matrix:
[[227 107]
 [ 22 714]]


Glove using SVC

In [ ]:
from sklearn.svm import SVC

# Average the word vectors for each sentence (and assign a vector of zeros if no embedding is found)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create an SVM classifier
svm = SVC()

# Fit the model to the oversampled training data
svm_model = svm.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = svm_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.823 / Recall: 0.829 / Accuracy: 0.76


Confusion Matrix:
[[203 131]
 [126 610]]


Glove using MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

# Average the word vectors for each sentence (and assign a vector of zeros if no embedding is found)
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

# Convert the data to numpy arrays
X_train_vect_avg = np.array(X_train_vect_avg)
X_test_vect_avg = np.array(X_test_vect_avg)

# Convert labels to numpy arrays
y_train_np = y_val_train.values.ravel()
y_test_np = pd.DataFrame(y_test).values.ravel()

# Perform oversampling using SMOTE
smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train_vect_avg, y_train_np)

# Create an MLP classifier with 1 hidden layer
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)

# Fit the model to the oversampled training data
mlp_model = mlp.fit(X_train_oversampled, y_train_oversampled)

# Use the trained model to make predictions on the test data
y_pred = mlp_model.predict(X_test_vect_avg)

# Evaluate the predictions of the model on the holdout test set
precision = precision_score(y_test_np, y_pred)
recall = recall_score(y_test_np, y_pred)
accuracy = accuracy_score(y_test_np, y_pred)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test_np, y_pred)

print('\n\nPrecision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))
print('\n\nConfusion Matrix:')
print(confusion_mat)



Precision: 0.847 / Recall: 0.859 / Accuracy: 0.796


Confusion Matrix:
[[220 114]
 [104 632]]
